# Data Preparation
## Getting the System Ready and Loading the data

In [18]:
import sys
from pathlib import Path

# make sure parent folder (project root) is on the path
project_root = Path.cwd().parent  # adjust if your notebook lives somewhere else
sys.path.insert(0, str(project_root))

from src.data_ingestion import load_sensor_data, load_profile_data, merge_data, save_cycles


raw_dir = "../data/raw"
profile_path = f"{raw_dir}/profile.txt"
output_csv = "../data/processed/hydraulic_cycles.csv"

sensors = load_sensor_data(raw_dir)
profiles = load_profile_data(profile_path)
df = merge_data(sensors, profiles)
save_cycles(df, output_csv)


## Understanding the Data

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2205 entries, 0 to 2204
Columns: 43685 entries, PS1_0 to stable_flag
dtypes: float64(43646), int64(39)
memory usage: 734.9 MB


In [5]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
PS1_0,2205.0,148.978467,3.684421,144.92,147.06,148.56,150.98,188.80
PS1_1,2205.0,148.977220,3.685462,144.92,147.10,148.56,150.98,188.88
PS1_2,2205.0,148.966916,3.685930,144.56,147.10,148.56,150.98,188.97
PS1_3,2205.0,148.647410,3.790893,144.44,146.18,148.34,150.87,188.84
PS1_4,2205.0,147.968898,4.023828,141.19,145.06,148.13,150.74,186.11
...,...,...,...,...,...,...,...,...
cooler_pct,2205.0,41.240816,42.383143,3.00,3.00,20.00,100.00,100.00
valve_pct,2205.0,90.693878,10.681802,73.00,80.00,100.00,100.00,100.00
pump_leak,2205.0,0.669388,0.817233,0.00,0.00,0.00,1.00,2.00
acc_pressure,2205.0,107.199546,16.435848,90.00,90.00,100.00,130.00,130.00


## Missing Value and Outlier Treatment